In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D,MaxPooling2D,AveragePooling2D,Input,Activation,Dense,Concatenate,BatchNormalization,Dropout,Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/train70000.csv")

In [ ]:
x_train = data.drop(['label','Unnamed: 0'],axis =1)

In [ ]:
y_train = data['label']

In [ ]:
x_train = np.array(x_train)
x_train.shape

In [ ]:
y_train = np.array(y_train)
y_train.shape

In [ ]:
x_train = x_train.astype('float')/255.

x_train = np.reshape(x_train,(42000,28,28,1))
x_train = np.pad(x_train, ((0, 0), (2, 2), (2, 2), (0, 0)), 'constant')

x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.10)

datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [ ]:
datagen.fit(x_train)

In [ ]:
plt.figure(figsize=(20,2))
for i in range(10):
  plt.subplot(1,10,i+1)
  plt.imshow(x_train[i].reshape(32,32), cmap='binary')
  plt.xticks([])
  plt.yticks([])
plt.show()

In [ ]:
a0 = keras.layers.Input(shape=(32,32,1))

z1 = Conv2D(4,(3,3),padding = 'same')(a0)
z1c = Concatenate()([a0,z1])
z1b = BatchNormalization()(z1c)
a1 = Activation('relu')(z1b)

z2 = Conv2D(4,(3,3),padding = 'same')(a1)
z2c = Concatenate()([a0,z1,z2])
z2b = BatchNormalization()(z2c)
a2 = Activation('relu')(z2b)


z3 = Conv2D(4,(3,3),padding = 'same')(a2)
z3c = Concatenate()([a0,z1,z2,z3])
z3b = BatchNormalization()(z3c)
a3 = Activation('relu')(z3b)


z4 = Conv2D(4,(3,3),padding = 'same')(a3)
z4c = Concatenate()([a0,z1,z2,z3,z4])
z4b = BatchNormalization()(z4)
a4 = Activation('relu')(z4b)


z5 = Conv2D(4,(3,3),padding = 'same')(a4)
z5c = Concatenate()([a0,z1,z2,z3,z4,z5])
z5b = BatchNormalization()(z5c)
a5 = Activation('relu')(z5b)
a5 = Dropout(0.5)(a5)


z7 = Conv2D(92,(5,5))(a5)
z7 = BatchNormalization()(z7)
a7 = Activation('relu')(z7)
a7 = AveragePooling2D(pool_size=(2,2),strides=2)(a7)
a7 = Dropout(0.5)(a7)

z8 = Conv2D(128,(5,5))(a7)
z8 = BatchNormalization()(z8)
a8 = Activation('relu')(z8)
a8 = AveragePooling2D(pool_size = (2,2),strides=2)(a8)
a8 = Dropout(0.5)(a8)

a_0 = keras.layers.GlobalAveragePooling2D()(a8)

z_1 = Dense(128)(a_0)
z_1 = BatchNormalization()(z_1)
a_1 = Activation('relu')(z_1)
a_1 = Dropout(0.4)(a_1)

z_2 = Dense(84)(a_1)
z_2 = BatchNormalization()(z_2)
a_2 = Activation('relu')(z_2)
a_1 = Dropout(0.4)(a_2)

z_3 = Dense(10)(a_2)
a_3 = Activation('softmax')(z_3)

In [ ]:
model = keras.models.Model(a0,a_3)

In [ ]:
model.save('mnist.h5')

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',optimizer=Adam(lr = 0.0003),metrics=['accuracy'])

In [ ]:
plot_model(model,to_file='LeNet-5.png',show_shapes=True,show_layer_names = True)

In [ ]:
history = model.fit(x_train,y_train,batch_size = 256,epochs = 150,verbose = 1,shuffle = True,validation_data=(x_val,y_val))

In [ ]:
 acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


epochs = range(len(acc))

plt.plot(epochs,acc,'r',label = 'training accuracy')
plt.plot(epochs,val_acc,'b',label = 'validation accuracy')
plt.legend(loc =0)
plt.figure()

plt.plot(epochs,loss,'r',label = 'training loss')
plt.plot(epochs,val_loss,'b',label = 'validation loss')
plt.legend(loc =0)
plt.figure()

In [ ]:
model.save('handwritten.h5')